# Scraping the data

## Do not run

In [129]:
import pandas as pd
import numpy as np
import gzip
import datetime
import requests
import sys
import os
from glob import glob
from StringIO import StringIO
from PIL import Image
import time

# import pickle

We first define parsing fucntions that take the gzipped JSON files from 
http://jmcauley.ucsd.edu/data/amazon/
to get pandas data frame use

    Pandas_df = getDF(path, N = Nuber_of_lines)
    


In [71]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path, N= int(1e9) ):
    print 'Getting first %d lines of ./%s' % (N,path)
    i = 0
    df = {}
    for d in parse(path):
        if i < N:
            df[i] = d
            i += 1
        else:
            break
    return pd.DataFrame.from_dict(df, orient='index')

In [131]:
paths  = [  'data/reviews/reviews_Baby_5.json.gz',
            'data/reviews/reviews_Beauty_5.json.gz',
            'data/reviews/reviews_Electronics_5.json.gz',
            'data/reviews/reviews_Sports_and_Outdoors_5.json.gz',
            'data/reviews/reviews_Musical_Instruments_5.json.gz']

paths_r  = ['data/reviews/ratings_Baby.csv',
            'data/reviews/ratings_Beauty.csv',
            'data/reviews/ratings_Electronics.csv',
            'data/reviews/ratings_Sports_and_Outdoors.csv',
            'data/reviews/ratings_Musical_Instruments.csv']

n = -1
path = paths[n]
path_r = paths_r[n]


In [132]:
## if slow
## first N lines: getDF(path, N)
df = getDF(path)

Getting first 1000000000 lines of ./data/reviews/reviews_Musical_Instruments_5.json.gz


In [133]:
# Define items by ASIN
items = df.asin.value_counts()
item_names = list(items.index.get_values())
print len(item_names)
items.head()


900


B003VWJ2K8    163
B0002E1G5C    143
B0002F7K7Y    116
B003VWKPHC    114
B0002H0A3S     93
Name: asin, dtype: int64

In [134]:
#### Save ######
# check the path !!!
path_to_save_count = path.replace('_5.json.gz', '.csv').replace('reviews_', 'counts_reviews_')
print path_to_save_count 

def query(question, default = 'No'):
    yes = {'yes','y', 'ye'}
    print question, ': '
    choice = raw_input('Enter for "No"').lower()
    if choice in yes:
        return True
    else:
        return False

print 'You are about to save {} items to the path :\n \t <{}>'.format(len(items),path_to_save_count)
    
if  os.path.exists(path_to_save_count):
    print '••• This path already exists •••'

    
if query('Do you want to save?'):
    items.to_csv( path_to_save_count)
    print "Saved"
else:
    print 'Not saved'


data/reviews/counts_reviews_Musical_Instruments.csv
You are about to save 900 items to the path :
 	 <data/reviews/counts_reviews_Musical_Instruments.csv>
••• This path already exists •••
Do you want to save? : 
Enter for "No"
Not saved


In [135]:
# Size of obeject and filenames

print  sys.getsizeof(df)/2**20, 'Mb'

filenames = [os.path.basename(i)[:-4] for i in glob('data/price_history/new/*.png',)]
print 'There are %d objects in the picture folder' % len(filenames)

8 Mb
There are 20157 objects in the picture folder


### Analyze ratings

In [136]:
path_r

'data/reviews/ratings_Musical_Instruments.csv'

In [137]:
columns = ['User', 'ASIN', 'Rating', 'Timestamp']
ratings_df = pd.read_csv(path_r, header=None, names = columns)
print sys.getsizeof(ratings_df)/2**20, 'Mb'
ratings_df.head()
# 

61 Mb


,User,ASIN,Rating,Timestamp
0,A1YS9MDZP93857,0006428320,3.0,1394496000
1,A3TS466QBAWB9D,0014072149,5.0,1370476800
2,A3BUDYITWUSIS7,0041291905,5.0,1381708800
3,A19K10Z0D2NTZK,0041913574,5.0,1285200000
4,A14X336IB4JD89,0201891859,1.0,1350432000


In [138]:
r = ratings_df.ASIN.value_counts()

In [139]:
# intersection if indexes such that 
#  reviews >4, ratings>50
print 'total items with 5+ reviews: %d, and 10+: %d, ratings>50: %d' % (len(items), len(items[items>10]),len(r[r>50]))
idx = items[items>4].index.intersection(r[r>50].index)
print 'intersection is ', len(items[idx])


total items with 5+ reviews: 900, and 10+: 270, ratings>50: 1326
intersection is  517


# Scrap

In [140]:
def getImage(asin,  tag =  'new', time_sleep = 5):
    if len(asin)==10:

        image_name = 'data/price_history/%s/%s.png' % (tag,asin)
        if os.path.exists('data/price_history/'+tag):
            url = r'https://charts.camelcamelcamel.com/us/%s/%s.png?force=1&zero=1&w=1000&h=500&desired=false&legend=1&ilt=1&tp=all&fo=0&lang=en'  \
            % (asin, tag)

            if not os.path.isfile(image_name):
#                 print  '... created \t',image_name
                try:
                    r = requests.get(url)
                    time.sleep(time_sleep+2*np.random.random())
                    if r.status_code == 200:
                        i = Image.open(StringIO(r.content))
                        i.save(image_name)
                    
                    else:
                        print 'status_code=', r.status_code
                except Exception as e:
                    print(e.message)

                
        else:
            print 'no such folder -->', tag
    else:
        print('Wrong ASIN provided')




#### THREE EXAMPLES OF DEFINING THE ITEM NAMES:
    

In [141]:
# (1) 
# from other directory:
# all scrapped items form new directory to get the amazon prices
item_names = glob('data/price_history/new/??????????.png')
item_names = [i[-14:-4] for i in item_names]
print len(item_names)
# (2)
# From itmes - list of unique tnries in df
item_names = list(items.index.get_values())
print len(item_names)
# (3) - prefferred
# From the intersection of items and ratings
# use idx defined above
item_names = list(idx.get_values())
print len(item_names)

20157
900
517


In [ ]:

counter = 0    ## for progress bar
tag = 'new'

for item in item_names:
    getImage(item,tag,time_sleep=2)
    counter +=1
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %s\t %d/%d" % ('='*(20*counter/len(item_names)), item, counter,len(item_names)))
    time.sleep(0.001)
    sys.stdout.flush()
    

[                    ] B004XNK7AI	 11/517

<div class="alert alert-block alert-info">
</div>


# Extra things

In [420]:
# ## thetracktor
# t = page.text
# r = re.compile('loadPrices\((\d+),')
# id = re.findall(r, t)

# json_url = r'https://thetracktor.com/ajax/prices/?id=%s&days=90' % id[0]
# print json_url
# # json_url = requests.get(json_url)
